In [1]:
import os
os.chdir('/home/victorhuang/projects/gtx/')
# os.chdir('/home/victorh/projects/gtx/')

import numpy as np
import mat73
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from numerical.op_props_util import *

In [2]:
# Spatial frequency
fx = [0, 0.05, 0.1, 0.15, 0.2, 0.25] 
fx = np.array(fx)

dt_data_path = 'data/20241118_data_splited.mat'
phantom_data_path = 'data/phantom_data.mat'
cylinder_data_path_r1 = 'data/cylinder/h1_r5_d0.mat'
cylinder_data_path_r2 = 'data/cylinder/h1_r4_d2.mat' 